In [27]:
import pickle
import numpy as np 
import gym 
import os
import load_policy
from model import get_trained_model
from sklearn.model_selection import train_test_split

import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
envname = "Ant-v2"

## Train Model for agent policy

In [7]:
with open(os.path.join('expert_data', envname + '.pkl'), 'rb') as f:
    expert_data = pickle.load(f)
# data for behavioural cloning
X = expert_data['observations']
Y = expert_data['actions']

In [8]:
model, metrics = get_trained_model(X, Y)

## Perform rollout and record agents observations

In [4]:
env = gym.make(envname)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [42]:
def rollout(n, max_steps):
    observations = []
    returns = []

    for i in range(n):
        obs = env.reset()
        done = False
        totalr = 0
        steps = 0
        while (not done) and steps <= max_steps:
            action = model.predict(obs[None, :])
            obs, r, done, _ = env.step(action)
            totalr += r
            steps += 1
            # record observation
            observations.append(obs)
        returns.append(totalr)
    return np.mean(returns), np.std(returns), observations

In [ ]:
_, _, observations = rollout(2, 1000)

In [23]:
len(observations)

2000

## Label the Agents Observations to the Experts Action

In [14]:
policy_fn = load_policy.load_policy('experts/'+ envname + '.pkl')

obs (1, 111) (1, 111)


In [24]:
actions = []
with tf.Session():
    tf.global_variables_initializer()
    
    for obs in observations:
        action = policy_fn(obs[None, :])
        actions.append(action)

In [37]:
X_train, X_val, Y_train, Y_val = train_test_split(np.asarray(observations),
                                                  np.asarray(actions), test_size=0.1)

metrics = model.fit(X_train, 
            Y_train,
            epochs=50,
            batch_size=64,
            verbose=0,
            validation_data=(X_val, Y_val))

In [40]:
metrics.history['loss'][-1], metrics.history['val_loss'][-1]

(0.005234863924690419, 0.00475783884525299)

# Rollouts

In [43]:
rmean, rstd, _ = rollout(2, 1000)

In [44]:
rmean, rstd

(101.44629968831195, 1.2519481924514366)